# Topog form stress using MOM025 DATA

Recent update 11 Sept 2017

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import os
import pickle
from numpy import math

import pandas as pd
import xarray as xr
from glob import glob
from mpl_toolkits.basemap import Basemap, shiftgrid


# Load Data

In [2]:
## Load data

%time
OceanFile1 = '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output000/ocean/ocean_grid.nc'
dsx1 = xr.open_dataset(OceanFile1, engine='netcdf4')
kmt = dsx1.kmt.isel(yt_ocean=217).astype(np.int64)  # "number of depth levels on t-grid"
depth = -dsx1.ht.isel(yt_ocean=217)   ## H
dxu = dsx1.dxu ## dxu

OceanFile2 =  '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output000/ocean/ocean_month.nc'
dsx2 = xr.open_dataset(OceanFile2, engine='netcdf4')
taux = dxu*dsx2.tau_x.mean('time')  ## tau_x
pbot = dsx2.pbot_t.mean('time').isel(yt_ocean=217)     ## p_bot
eta0 = dsx2.eta_t.mean('time') ## \eta
eta = dsx2.sea_level.mean('time') ## sea_level

OceanFile3 =  '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output000/ocean/ocean.nc'
dsx3 = xr.open_dataset(OceanFile3, engine='netcdf4')
rho = dsx3.rho.mean('time').isel(yt_ocean=217)        ## \rho in-situ
dz = dsx3.dzt.mean('time').isel(yt_ocean=217)      ## t-cell thickness
g = 9.8

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


/g/data3/hh5/public/apps/miniconda3/envs/analysis27/lib/python2.7/site-packages/xarray/conventions.py:393: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  result = decode_cf_datetime(example_value, units, calendar)
/g/data3/hh5/public/apps/miniconda3/envs/analysis27/lib/python2.7/site-packages/xarray/conventions.py:412: RuntimeWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using dummy netCDF4.datetime objects instead, reason: dates out of range
  calendar=self.calendar)


## Step 1: 
## define water thickness as h: $h = -\int dz$
## define bottom pressure as p: $p = \int \rho*g*dz$

In [156]:
h = -dz.sum('st_ocean')

dp = rho*g*dz
p = dp.sum('st_ocean')

## Step 2:
## Calculate $\sum p*\Delta z$

In [175]:
z=np.zeros(kmt.shape).astype(np.int64) 
z[:]=kmt[:].values-1
dp = 0

pb=np.zeros(p.shape)
pb[:]=p[:].values

for x in range(1386,1387):
    dkmt1=kmt.isel(xt_ocean=x)-kmt.isel(xt_ocean=x-1)
    if dkmt1==0:
        dz1=dz[z[x],x-1]
        dz2=dz[z[x],x]
        if (dz1-dz2)<0:
            rho1=rho[z[x],x]
            dp=dp+(pb[x]*(dz2-dz1)-g*rho1*(dz2-dz1)**2/2)
        elif (dz1-dz2)>0:
            rho1=rho[z[x-1],x-1]
            dp=dp-(pb[x-1]*(dz1-dz2)-g*rho1*(dz2-dz1)**2/2)
    elif dkmt1==1:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x] - dz[z[x]-1,x-1]
        dp=dp+(pb[x]*(dz1+dz2)-g*rho1*dz1*dz2-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2)
    elif dkmt1==-1:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1] - dz[z[x-1]-1,x]
        dp=dp-(pb[x-1]*(dz1+dz2)-g*rho1*dz1*dz2-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2)
    elif dkmt1==2:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]- dz[z[x]-2,x-1]
        dp=dp+(pb[x]*(dz1+dz2+dz3)-g*rho1*dz1*(dz2+dz3)-g*rho2*dz2*(dz3)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2)
    elif dkmt1==-2:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]- dz[z[x-1]-2,x]
        dp=dp-(pb[x-1]*(dz1+dz2+dz3)-g*rho1*dz1*(dz2+dz3)-g*rho2*dz2*(dz3)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2)       
    elif dkmt1==3:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]- dz[z[x]-3,x-1]
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4)-g*rho1*dz1*(dz2+dz3+dz4)-g*rho2*dz2*(dz3+dz4)-g*rho3*dz3*(dz4)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2)        
    elif dkmt1==-3:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]- dz[z[x-1]-3,x]
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4)-g*rho1*dz1*(dz2+dz3+dz4)-g*rho2*dz2*(dz3+dz4)-g*rho3*dz3*(dz4)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2)
    elif dkmt1==4:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]        
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]- dz[z[x]-4,x-1]
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5)-g*rho1*dz1*(dz2+dz3+dz4+dz5)-g*rho2*dz2*(dz3+dz4+dz5)-g*rho3*dz3*(dz4+dz5)-g*rho4*dz4*(dz5)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2)
    elif dkmt1==-4:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]        
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]- dz[z[x-1]-4,x]
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5)-g*rho1*dz1*(dz2+dz3+dz4+dz5)-g*rho2*dz2*(dz3+dz4+dz5)-g*rho3*dz3*(dz4+dz5)-g*rho4*dz4*(dz5)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2)
    elif dkmt1==5:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]    
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]- dz[z[x]-5,x-1]
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6)-g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6)-g*rho2*dz2*(dz3+dz4+dz5+dz6)-g*rho3*dz3*(dz4+dz5+dz6)-g*rho4*dz4*(dz5+dz6)-g*rho5*dz5*(dz6)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2)         
    elif dkmt1==-5:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]- dz[z[x-1]-5,x]
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6)-g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6)-g*rho2*dz2*(dz3+dz4+dz5+dz6)-g*rho3*dz3*(dz4+dz5+dz6)-g*rho4*dz4*(dz5+dz6)-g*rho5*dz5*(dz6)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2)         
    elif dkmt1==-7:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]
        rho7=rho[z[x-1]-6,x-1]
        dz7=dz[z[x-1]-6,x-1]        
        rho8=rho[z[x-1]-7,x-1]
        dz8=dz[z[x-1]-7,x-1]- dz[z[x-1]-7,x]
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8)-g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8)-g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8)-g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8)-g*rho4*dz4*(dz5+dz6+dz7+dz8)-g*rho5*dz5*(dz6+dz7+dz8)-g*rho6*dz6*(dz7+dz8)-g*rho7*dz7*(dz8)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2-g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2)         
    elif dkmt1==9:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]    
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]
        rho7=rho[z[x]-6,x]
        dz7=dz[z[x]-6,x] 
        rho8=rho[z[x]-7,x]
        dz8=dz[z[x]-7,x]
        rho9=rho[z[x]-8,x]
        dz9=dz[z[x]-8,x]        
        rho10=rho[z[x]-9,x]
        dz10=dz[z[x]-9,x]- dz[z[x]-9,x-1]
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)-g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)-g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)-g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8+dz9+dz10)-g*rho4*dz4*(dz5+dz6+dz7+dz8+dz9+dz10)-g*rho5*dz5*(dz6+dz7+dz8+dz9+dz10)-g*rho6*dz6*(dz7+dz8+dz9+dz10)-g*rho7*dz7*(dz8+dz9+dz10)-g*rho8*dz8*(dz9+dz10)-g*rho9*dz9*(dz10)-g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2-g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2-g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2-g*rho9*(dz9)**2/2-g*rho10*(dz10)**2/2)                                   
       

In [176]:
dkmt1

<xarray.DataArray 'kmt' ()>
array(-3)
Coordinates:
    yt_ocean  float64 -57.23
    geolat_t  float64 -57.23

In [177]:
dp

<xarray.DataArray ()>
array(-19696360825.294518)
Coordinates:
    xt_ocean  float64 66.38
    yt_ocean  float64 -57.23
    st_ocean  float64 3.3e+03

In [190]:
dz4

<xarray.DataArray 'dzt' ()>
array(36.831298828125)
Coordinates:
    yt_ocean  float64 -57.23
    st_ocean  float64 3.3e+03

In [191]:
z=42
x=1386

-(pb[x-1]*(dz[z,x-1]+dz[z-1,x-1]+dz[z-2,x-1]+dz[z-3,x-1]-dz[z-3,x])-g*rho[z,x-1]*dz[z,x-1]*(dz[z-1,x-1]+dz[z-2,x-1]+dz[z-3,x-1]-dz[z-3,x])-g*rho[z-1,x-1]*dz[z-1,x-1]*(dz[z-2,x-1]+dz[z-3,x-1]-dz[z-3,x])-g*rho[z-2,x-1]*dz[z-2,x-1]*(dz[z-3,x-1]-dz[z-3,x])-g*rho[z,x-1]*(dz[z,x-1])**2/2-g*rho[z-1,x-1]*(dz[z-1,x-1])**2/2-g*rho[z-2,x-1]*(dz[z-2,x-1])**2/2-g*rho[z-3,x-1]*(dz[z-3,x-1]-dz[z-3,x])**2/2)

<xarray.DataArray ()>
array(-19696360825.294518)
Coordinates:
    yt_ocean  float64 -57.23
    xt_ocean  float64 66.38
    st_ocean  float64 3.3e+03

In [162]:
dz[z,x]

<xarray.DataArray 'dzt' ()>
array(36.2744140625)
Coordinates:
    xt_ocean  float64 70.88
    yt_ocean  float64 -57.23
    st_ocean  float64 1.498e+03

In [116]:
dz2

<xarray.DataArray 'dzt' ()>
array(148.01780700683594)
Coordinates:
    xt_ocean  float64 71.88
    yt_ocean  float64 -57.23
    st_ocean  float64 1.875e+03

In [61]:
dz1

<xarray.DataArray 'dzt' ()>
array(205.3374786376953)
Coordinates:
    xt_ocean  float64 67.88
    yt_ocean  float64 -57.23
    st_ocean  float64 2.679e+03